문서를 업로드

렝체인 사용준비

In [ ]:
!pip install -q langchain openai tiktoken chromadb

In [ ]:
import os

os.environ["OPENAI_API_KEY"]="gpt개인키"

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

여러가지 파일 로드시

*   loader = DirectoryLoder('./test',
glob="*txt", loader_cls=TextLoader)

*   glob에는 확장자, './test'는 파일 경로, TextLoader를 사용해서 로드 하겠다.

*   articles 폴더 안에 있는 txt 확장자를 가진 모든 파일 로드

파일 하나 로드시


*   loader = TextLoader('single_text_file.txt')
*   괄호 안에는 파일 이름



In [ ]:
#loader = DirectoryLoder('./test', glob="*txt", loader_cls=TextLoader)
#loader = TextLoader('single_text_file.txt')
#둘중 필요한 쪽으로 해서 주석 지울것(양자택일)

documents = loader.load()

len(documents)
#바로 위의 코드는 몇개의 문서가 로드되었는지 알려주는 코드

### split(나누기)하는 이유
문서의 분량이 많을 수 있기때문에 chatGPT한테 요청을 보낼 때 토큰수를 초과할 수도 있음.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#chunk_size=1000는 1000글자씩 분할해달라
#chunk_overlap=200은 끊기는 걸 방지하기 위해서 겹치는 부분 200자
texts = text_splitter.split_documents(documents)

len(texts)
#몇가지로 나뉘어지는지 알려주는 코드

In [ ]:
texts[2:4]
#잘 나눠졌는지 확인하기 위해서 순서상 2번째랑 3번째로 나뉘어진게 뭔지 출력

## 벡터 데이터 베이스 사용

Chroma DB를 사용한다고 가정


*   Text를 Embbedings(벡터형태로 저장)
*   Chroma DB는 무료 벡터DB, 사용법이 간단




In [ ]:
persist_directory = 'db'
#데이터들을 db라는 폴더에 저장하려고 설정한 것

embedding = OpenAIEmbeddings()
#chatGPT를 사용하기때문에 OpenAIEmbedding을 함

vectordb = Chroma.from_documents(
    documents=texts,                    #우리가 미리 나눠둔 texts(문서 나눈 것)들을
    embedding=embedding,                #OpenAIEmbedding을 사용해
    persist_directory=persist_directory #db라는 폴더에 저장을 하겠다.
)

#실행시 코랩 옆 폴더에서 db라는 폴더가 만들어지는 것을 확인할 수 있음

In [ ]:
#초기화
vectordb.persist()
vectordb = None

In [ ]:
#실제로 다 만들어졌을 경우에는 이코드만 실행해도 괜찮을 것
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

## 벡터db 사용방법은 retriever를 통해서

retriever = 연관있는 문서 출력해주는 함수(문서검색기)

In [ ]:
retriever = vectordb.as_retriever()   #연관있는 문서가 몇개든 모두반환
retriever = vectordb.as_retriever(search_kwargs={"k":3}) #연관있는 문서 3개만 반환
                                                    #k값 조절해서 반환수 조절가능

In [ ]:
docs = retriever.get_relevant_documents("(질문)")
#예시 : docs = retriever.get_relevant_documents("이종혁교수님에 대해 알려줘")

for doc in docs :
  print(doc.metadata["source"])

#결과로 질문과 관련있는 연관된 문서가 무엇인지 출력

## chain 만들기

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),                     #만들 때 사용하는 대규모 언어모델은 ChatGPT
    chain_type="stuff",               #stuff라는 타입사용
    retriever=retriever,
    return_source_documents=True      #True는 원본문서를 반환하도록 하는 것
)

In [ ]:
def process_llm_response(llm_response) :
  print(llm_response['result'])           #chatGPT가 요약한 결과물
  print('\n\nSources:')
  for source in llm_response["source_documents"]:
    print(source.metadata['source'])

## 질문하기

*테스트 성공시 질문 내용을 받는 형식으로 코드 수정 필요

In [ ]:
query = "(질문 내용)"
llm_response = qa_chain(query)      #chain만들기 첫번째로 정의 함수
process_llm_response(llm_response)  #chain만들기 두번째로 정의한 함수

In [ ]:
#llm_response
#이 코드 사용시 질문 내용, GPT의 답변, 문서 어느 부분에서 찾았는지 상세하게 나옴